In [16]:
import random
import numpy as np

# Configuração do Problema
# Ordens de Serviço (OS): (id, tipo, nível necessário, prazo)
ordens_servico = [
    (1, "mecanica", 3, 2),
    (2, "eletrica", 2, 1),
    (3, "mecanica", 2, 3),
    (4, "eletrica", 1, 2),
    (5, "mecanica", 1, 1),
]

# Operadores: (id, habilidades, nível máximo, disponibilidade em dias)
operadores = [
    (1, ["mecanica"], 3, 5),
    (2, ["eletrica"], 2, 4),
    (3, ["mecanica", "eletrica"], 2, 3),
]

In [17]:
# Representação do Cromossomo
def gerar_cromossomo():
    return [random.choice(range(len(operadores))) for _ in ordens_servico]

In [18]:
# Função de Fitness
def calcular_fitness(cromossomo):
    fitness = 0

    for os_index, operador_index in enumerate(cromossomo):
        os_tipo, os_nivel, os_prazo = ordens_servico[os_index][1:]
        op_habilidades, op_nivel_max, op_disponibilidade = operadores[operador_index][1:]

        # Penalidade se o operador não tem a habilidade necessária
        if os_tipo not in op_habilidades:
            fitness -= 10

        # Penalidade se o operador não atinge o nível necessário
        if os_nivel > op_nivel_max:
            fitness -= 5

        # Penalidade se o operador não tem disponibilidade suficiente
        if os_prazo > op_disponibilidade:
            fitness -= 5

        # Recompensa se tudo está correto
        if os_tipo in op_habilidades and os_nivel <= op_nivel_max and os_prazo <= op_disponibilidade:
            fitness += 10

    return fitness


In [19]:

# Operadores Genéticos
def selecao_torneio(populacao, fitnesses, tamanho=3):
    selecionados = random.sample(list(zip(populacao, fitnesses)), tamanho)
    return max(selecionados, key=lambda x: x[1])[0]


In [20]:
def crossover(pai1, pai2):
    ponto = random.randint(1, len(pai1) - 1)
    filho1 = pai1[:ponto] + pai2[ponto:]
    filho2 = pai2[:ponto] + pai1[ponto:]
    return filho1, filho2



In [21]:
def mutacao(cromossomo, taxa_mutacao=0.1):
    for i in range(len(cromossomo)):
        if random.random() < taxa_mutacao:
            cromossomo[i] = random.choice(range(len(operadores)))
    return cromossomo



In [22]:
# Algoritmo Genético
def algoritmo_genetico(geracoes=50, populacao_tamanho=20, taxa_mutacao=0.1):
    print(f"Iniciando algoritmo genético com {geracoes} gerações")
    melhor_fitness_historico = []
    melhor_fitness_global = float('-inf') 
    
    # Gerar população inicial
    populacao = [gerar_cromossomo() for _ in range(populacao_tamanho)]

    for geracao in range(geracoes):
        # Avaliar a população
        fitnesses = [calcular_fitness(cromo) for cromo in populacao]
        melhor_fitness_atual = max(fitnesses)
        melhor_fitness_historico.append(melhor_fitness_atual)

        if melhor_fitness_atual > melhor_fitness_global:
            melhor_fitness_global = melhor_fitness_atual
            print(f"Geração {geracao}: Melhor fitness = {melhor_fitness_atual}")
    
        # Seleção e reprodução
        nova_populacao = []
        while len(nova_populacao) < populacao_tamanho:
            pai1 = selecao_torneio(populacao, fitnesses)
            pai2 = selecao_torneio(populacao, fitnesses)
            filho1, filho2 = crossover(pai1, pai2)
            nova_populacao.extend([mutacao(filho1, taxa_mutacao), mutacao(filho2, taxa_mutacao)])

        populacao = nova_populacao[:populacao_tamanho]

    # Obter a melhor solução
    fitnesses = [calcular_fitness(cromo) for cromo in populacao]
    melhor_indice = np.argmax(fitnesses)
    melhor_solucao = populacao[melhor_indice]

    return melhor_solucao, fitnesses[melhor_indice]

In [23]:

def imprimir_resultado(solucao, fitness):
    print("\n=== Resultado da Otimização ===")
    print(f"Melhor Fitness: {fitness}")
    print(f"Melhor Solucao: {solucao}")
    print("\nDesignação de tarefas:")

    print("-" * 110)
    for os_index, operador_index in enumerate(solucao):
        os = ordens_servico[os_index]
        op = operadores[operador_index]
        # Ordens de Serviço (OS): (id, tipo, nível necessário, prazo)
        print(f"OS {os[0]:2d} ({os[1]:8s}, nível {os[2]}, prazo{os[3]:2d}) → Operador {op[0]} "
              f"(nível máx: {op[2]} - habilidades: {', '.join(op[1])} - disp.: {op[3]})")
    print("-" * 110)

# Execução do Algoritmo
melhor_solucao, melhor_fitness = algoritmo_genetico()
imprimir_resultado(melhor_solucao, melhor_fitness)

Iniciando algoritmo genético com 50 gerações
Geração 0: Melhor fitness = 50

=== Resultado da Otimização ===
Melhor Fitness: 50
Melhor Solucao: [0, 2, 0, 1, 0]

Designação de tarefas:
--------------------------------------------------------------------------------------------------------------
OS  1 (mecanica, nível 3, prazo 2) → Operador 1 (nível máx: 3 - habilidades: mecanica - disp.: 5)
OS  2 (eletrica, nível 2, prazo 1) → Operador 3 (nível máx: 2 - habilidades: mecanica, eletrica - disp.: 3)
OS  3 (mecanica, nível 2, prazo 3) → Operador 1 (nível máx: 3 - habilidades: mecanica - disp.: 5)
OS  4 (eletrica, nível 1, prazo 2) → Operador 2 (nível máx: 2 - habilidades: eletrica - disp.: 4)
OS  5 (mecanica, nível 1, prazo 1) → Operador 1 (nível máx: 3 - habilidades: mecanica - disp.: 5)
--------------------------------------------------------------------------------------------------------------
